# Association Analysis

In [6]:
import pandas as pd
from apyori import apriori
import sqlite3
from mlxtend.frequent_patterns import apriori, association_rules 

## Connect with sqlite3 db

In [7]:
conn = sqlite3.connect('hatch.db')

In [8]:
df = pd.read_sql_query('select * from hatch_bill_data',con=conn)

In [9]:
df.head()

,S_No,BillDate_and_Time,BillNo,Tbl_,Pax,KotNo,KOTDate_and_Time,Steward,Classification,MenuGroup,...,Qty,Amount,Discount,CGS_2_5,SGS_2_5,Total,UserID,PayMode,Tips,Restaurant
0,1,2019-04-02T21:37:00,3,3,2,7,2019-04-02T20:31:00,Ganesh,Liquor,Vodka,...,2.0,280.0,0.0,0.000,0.000,280.00,NAGESH,None,None,The Estate
1,1,2019-04-02T21:37:00,3,3,2,7,2019-04-02T20:31:00,Ganesh,Liquor,Scotch/ Bourbon,...,2.0,310.0,0.0,0.000,0.000,310.00,NAGESH,None,None,The Estate
2,1,2019-04-02T21:37:00,3,3,2,7,2019-04-02T20:31:00,Ganesh,Others,Soft Beverages,...,1.0,20.0,0.0,0.500,0.500,21.00,NAGESH,None,None,The Estate
3,1,2019-04-02T21:37:00,3,3,2,7,2019-04-02T20:31:00,Ganesh,Soft Drink,Soft Beverages,...,1.0,50.0,0.0,1.250,1.250,52.50,NAGESH,None,None,The Estate
4,1,2019-04-02T21:37:00,3,3,2,7,2019-04-02T20:31:00,Ganesh,Food,Vegetarian Starters,...,1.0,175.0,0.0,4.375,4.375,183.76,NAGESH,None,None,The Estate


In [12]:
df.ItemName = df.ItemName.str.lower()

In [14]:
len(df.ItemName.unique())

404

In [7]:
q['Home Delivery']

BillDate_and_Time    BillNo
2019-04-02T22:43:00  2         [Nati Koli Curry, AKKI ROTTI, KALMI KABAB, Car...
2019-04-06T20:10:00  3         [HOT AND SOUR VEG, PERI PERI CHICKEN WINGS 1/2...
2019-04-07T20:58:00  4         [MALNAD PULAV - MUTTON, Chicken Kabab, NEER DO...
2019-04-10T20:30:00  5         [MANCHOW SOUP VEG, HARYALI PANEER KABAB, Spicy...
2019-04-12T19:44:00  6                           [RAGI MUDDE, AVARE KAALU GOJJU]
2019-04-13T21:33:00  7         [Mushroom & Coconut Soup, HOT AND SOUR VEG, PE...
2019-04-13T22:49:00  8                               [Mutton Nalli, SEEKH KABAB]
2019-04-14T12:49:00  9         [KORI ROTI, MANGALORE CHICKEN CURRY, NEER DOSA...
2019-04-15T19:27:00  10                                    [MASAPPU, RAGI MUDDE]
2019-04-15T22:04:00  11        [Coorgi Pork Fry, KANE FISH CURRY, NEER DOSA, ...
2019-04-20T13:14:00  12               [Corn & Cheese Ball, FRIED RICE - CHICKEN]
2019-04-23T21:45:00  13        [BANG BANG CHICKEN, PERI PERI CHICKEN WINGS 1 ...


In [8]:
records = q.to_list()

In [9]:
len(records)

1776

In [23]:
association_rules = apriori(records, min_support=0.006, min_confidence=0.5, min_lift=0, max_length=4)

In [24]:
association_rules

<generator object apriori at 0x11132ef68>

In [25]:
rules = list(association_rules)

In [29]:
g = rules[0]

In [31]:
m = g.items

In [36]:
list(m)

['100 Pipers', 'BOTTLED WATER']

In [35]:
{1,2,3}[0]

TypeError: 'set' object does not support indexing

In [21]:
df_the_estate = df[df.Restaurant == 'The Estate'].groupby(['BillDate_and_Time', 'ItemName'])['Qty'].sum().unstack().reset_index().fillna(0).set_index('BillDate_and_Time')
    
    

In [22]:
def hot_encode(x): 
    if(x<= 0): 
        return 0
    if(x>= 1): 
        return 1

In [35]:
df_the_estate = df_the_estate.applymap(hot_encode)

In [36]:
c = df_the_estate.loc['2019-04-02T21:37:00']
c[c > 0]

ItemName
akki rotti              1
almond cake             1
aloo mutter tikki       1
avare kaalu gojju       1
black & white           1
bottled water           1
chilli cheese toast     1
half rice               1
haryali paneer kabab    1
neer dosa               1
smirnoff orange         1
sprite                  1
Name: 2019-04-02T21:37:00, dtype: int64

In [37]:
df.query("BillDate_and_Time == '2019-04-02T21:37:00' and Restaurant == 'The Estate'")['ItemName']

0          smirnoff orange
1            black & white
2            bottled water
3                   sprite
4      chilli cheese toast
5     haryali paneer kabab
6        aloo mutter tikki
7            black & white
8                neer dosa
9               akki rotti
10       avare kaalu gojju
11               half rice
12             almond cake
Name: ItemName, dtype: object

In [43]:
len(df_the_estate)

221

In [61]:
frq_items = apriori(df_the_estate, min_support = 0.05, use_colnames = True)

In [62]:
frq_items

,support,itemsets
0,0.054299,(100 pipers)
1,0.271493,(akki rotti)
2,0.140271,(avare kaalu gojju)
3,0.108597,(black & white)
4,0.533937,(bottled water)
5,0.076923,(budweiser)
6,0.058824,(caesar salad)
7,0.095023,(chicken biryani)
8,0.072398,(chicken ghee roast)
9,0.099548,(cigarettes - 10 nos)


KeyError: True